<a href="https://colab.research.google.com/github/Kiran-01/Projects/blob/master/LSTM_Time_Series_Forecasting_using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# for reproducibility of the results, lets fix a seed function
np.random.seed(1234)

In [ ]:
df = pd.read_csv('international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)

In [ ]:
df.head()

,International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60
0,112
1,118
2,132
3,129
4,121


In [ ]:
df = df.values
df = df.astype('float32')

In [ ]:
#Normalize the data
scaler = MinMaxScaler(feature_range=(0,1))
df = scaler.fit_transform(df)

In [ ]:
df

array([[0.01544401],
       [0.02702703],
       [0.05405405],
       [0.04826255],
       [0.03281853],
       [0.05984557],
       [0.08494207],
       [0.08494207],
       [0.06177607],
       [0.02895753],
       [0.        ],
       [0.02702703],
       [0.02123553],
       [0.04247104],
       [0.07142857],
       [0.05984557],
       [0.04054055],
       [0.08687258],
       [0.12741312],
       [0.12741312],
       [0.10424709],
       [0.05598456],
       [0.01930502],
       [0.06949806],
       [0.07915059],
       [0.08880308],
       [0.14285713],
       [0.11389962],
       [0.13127413],
       [0.14285713],
       [0.18339768],
       [0.18339768],
       [0.15444016],
       [0.11196911],
       [0.08108109],
       [0.1196911 ],
       [0.12934363],
       [0.14671814],
       [0.17181468],
       [0.14864865],
       [0.15250966],
       [0.22007722],
       [0.24324325],
       [0.26640925],
       [0.2027027 ],
       [0.16795367],
       [0.13127413],
       [0.173

In [ ]:
train_size = int(len(df)*0.67)

In [ ]:
print(train_size)

96


In [ ]:
test_size = len(df) - train_size

In [ ]:
print(test_size)

48


In [ ]:
train, test = df[0: train_size, :], df[train_size: len(df), :]

In [ ]:
print(len(train), len(test))

96 48


In [ ]:
# Create dataset matrix

def create_dataset(df, look_back = 1):
  dataX, dataY = [], []
  for i in range(len(df)- look_back - 1):
    a = df[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(df[i+look_back, 0])
  return np.array(dataX), np.array(dataY)

In [ ]:
dataX

In [ ]:
# reshape dataset X = current time, Y = future time
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
print(trainX)

[[[0.01544401]]

 [[0.02702703]]

 [[0.05405405]]

 [[0.04826255]]

 [[0.03281853]]

 [[0.05984557]]

 [[0.08494207]]

 [[0.08494207]]

 [[0.06177607]]

 [[0.02895753]]

 [[0.        ]]

 [[0.02702703]]

 [[0.02123553]]

 [[0.04247104]]

 [[0.07142857]]

 [[0.05984557]]

 [[0.04054055]]

 [[0.08687258]]

 [[0.12741312]]

 [[0.12741312]]

 [[0.10424709]]

 [[0.05598456]]

 [[0.01930502]]

 [[0.06949806]]

 [[0.07915059]]

 [[0.08880308]]

 [[0.14285713]]

 [[0.11389962]]

 [[0.13127413]]

 [[0.14285713]]

 [[0.18339768]]

 [[0.18339768]]

 [[0.15444016]]

 [[0.11196911]]

 [[0.08108109]]

 [[0.1196911 ]]

 [[0.12934363]]

 [[0.14671814]]

 [[0.17181468]]

 [[0.14864865]]

 [[0.15250966]]

 [[0.22007722]]

 [[0.24324325]]

 [[0.26640925]]

 [[0.2027027 ]]

 [[0.16795367]]

 [[0.13127413]]

 [[0.17374519]]

 [[0.17760617]]

 [[0.17760617]]

 [[0.25482625]]

 [[0.25289574]]

 [[0.24131274]]

 [[0.26833975]]

 [[0.3088803 ]]

 [[0.32432434]]

 [[0.25675675]]

 [[0.20656371]]

 [[0.14671814]

In [ ]:
print(testX)

[[[0.4073359 ]]

 [[0.3803089 ]]

 [[0.48648646]]

 [[0.47104248]]

 [[0.484556  ]]

 [[0.6138996 ]]

 [[0.6969112 ]]

 [[0.70077217]]

 [[0.57915056]]

 [[0.46911195]]

 [[0.38803086]]

 [[0.44787642]]

 [[0.45559844]]

 [[0.4131274 ]]

 [[0.4980695 ]]

 [[0.47104248]]

 [[0.49999997]]

 [[0.6389961 ]]

 [[0.7471043 ]]

 [[0.7741313 ]]

 [[0.57915056]]

 [[0.492278  ]]

 [[0.3976834 ]]

 [[0.44980696]]

 [[0.49420848]]

 [[0.45945945]]

 [[0.5830116 ]]

 [[0.5637065 ]]

 [[0.61003864]]

 [[0.71042466]]

 [[0.8571429 ]]

 [[0.8783784 ]]

 [[0.69305015]]

 [[0.5849421 ]]

 [[0.4980695 ]]

 [[0.58108103]]

 [[0.6042471 ]]

 [[0.554054  ]]

 [[0.60810804]]

 [[0.6891892 ]]

 [[0.71042466]]

 [[0.8320464 ]]

 [[1.        ]]

 [[0.96911204]]

 [[0.7799227 ]]

 [[0.6891892 ]]]


In [ ]:
# LSTM (RNN) model

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape = (1, look_back)))
model.add(Dense(1))

model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [ ]:
model.fit(trainX, trainY, batch_size = 1,verbose = 2)

94/94 - 0s - loss: 0.0268


In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
print(trainPredict)

[[0.17396227]
 [0.17351857]
 [0.17250296]
 [0.17271827]
 [0.17329863]
 [0.17228895]
 [0.17137645]
 [0.17137645]
 [0.1722179 ]
 [0.17344512]
 [0.17456165]
 [0.17351857]
 [0.17373979]
 [0.17293485]
 [0.17186478]
 [0.17228895]
 [0.17300731]
 [0.17130727]
 [0.16988796]
 [0.16988796]
 [0.17069113]
 [0.1724315 ]
 [0.17381382]
 [0.17193513]
 [0.17158487]
 [0.17123823]
 [0.1693643 ]
 [0.17035392]
 [0.16975617]
 [0.1693643 ]
 [0.16803497]
 [0.16803497]
 [0.16897777]
 [0.17042106]
 [0.17151526]
 [0.17015335]
 [0.16982199]
 [0.16923489]
 [0.16840804]
 [0.16917038]
 [0.16904183]
 [0.16688938]
 [0.16619414]
 [0.16552103]
 [0.16742522]
 [0.1685336 ]
 [0.16975617]
 [0.16834548]
 [0.16822083]
 [0.16822083]
 [0.16585481]
 [0.16591099]
 [0.16625124]
 [0.16546592]
 [0.16434479]
 [0.16393577]
 [0.16579881]
 [0.16730507]
 [0.16923489]
 [0.1679118 ]
 [0.1677282 ]
 [0.16872308]
 [0.16591099]
 [0.16636589]
 [0.16596732]
 [0.16434479]
 [0.16249135]
 [0.1629098 ]
 [0.16460551]
 [0.16625124]
 [0.16778925]
 [0.16

In [ ]:
print(testPredict)

[[0.16190958]
 [0.16253722]
 [0.16025072]
 [0.16055337]
 [0.160288  ]
 [0.15814483]
 [0.15714742]
 [0.15710823]
 [0.15865001]
 [0.1605919 ]
 [0.16235472]
 [0.16102645]
 [0.1608662 ]
 [0.1617791 ]
 [0.16003048]
 [0.16055337]
 [0.15999433]
 [0.1578122 ]
 [0.15668696]
 [0.15648323]
 [0.15865001]
 [0.16013986]
 [0.16213018]
 [0.16098616]
 [0.16010325]
 [0.16078702]
 [0.15859133]
 [0.15889117]
 [0.1581984 ]
 [0.15701292]
 [0.15604939]
 [0.15598455]
 [0.15718731]
 [0.1585622 ]
 [0.16003048]
 [0.15862055]
 [0.15827994]
 [0.15904711]
 [0.1582254 ]
 [0.15722778]
 [0.15701292]
 [0.15615019]
 [0.15596859]
 [0.15591586]
 [0.15644358]
 [0.15722778]]


In [ ]:
# Reverse the predicted values to actual values

trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)

trainY = scaler.inverse_transform([trainY])
testY = scaler.inverse_transform([testY])

In [ ]:
print(trainPredict, trainY)

[[194.11246]
 [193.88261]
 [193.35654]
 [193.46806]
 [193.76869]
 [193.24568]
 [192.773  ]
 [192.773  ]
 [193.20888]
 [193.84457]
 [194.42294]
 [193.88261]
 [193.99721]
 [193.58026]
 [193.02596]
 [193.24568]
 [193.61778]
 [192.73717]
 [192.00197]
 [192.00197]
 [192.41801]
 [193.31952]
 [194.03555]
 [193.0624 ]
 [192.88097]
 [192.7014 ]
 [191.73071]
 [192.24333]
 [191.9337 ]
 [191.73071]
 [191.04211]
 [191.04211]
 [191.53049]
 [192.2781 ]
 [192.84491]
 [192.13943]
 [191.9678 ]
 [191.66368]
 [191.23537]
 [191.63026]
 [191.56367]
 [190.44872]
 [190.08856]
 [189.7399 ]
 [190.72626]
 [191.3004 ]
 [191.9337 ]
 [191.20296]
 [191.1384 ]
 [191.1384 ]
 [189.9128 ]
 [189.9419 ]
 [190.11815]
 [189.71135]
 [189.1306 ]
 [188.91872]
 [189.88379]
 [190.66403]
 [191.66368]
 [190.97832]
 [190.88321]
 [191.39856]
 [189.9419 ]
 [190.17754]
 [189.97107]
 [189.1306 ]
 [188.17052]
 [188.38728]
 [189.26567]
 [190.11815]
 [190.91483]
 [190.11815]
 [189.7399 ]
 [190.00032]
 [189.05054]
 [188.99757]
 [188.9712 ]

In [ ]:
print(testPredict, testY)

[[187.86917]
 [188.19427]
 [187.00987]
 [187.16664]
 [187.02919]
 [185.91902]
 [185.40237]
 [185.38206]
 [186.18071]
 [187.1866 ]
 [188.09976]
 [187.4117 ]
 [187.32869]
 [187.80157]
 [186.8958 ]
 [187.16664]
 [186.87706]
 [185.74672]
 [185.16385]
 [185.05832]
 [186.18071]
 [186.95245]
 [187.98343]
 [187.39082]
 [186.93347]
 [187.28767]
 [186.15031]
 [186.30563]
 [185.94676]
 [185.33269]
 [184.83359]
 [184.8    ]
 [185.42303]
 [186.13522]
 [186.8958 ]
 [186.16545]
 [185.989  ]
 [186.3864 ]
 [185.96075]
 [185.444  ]
 [185.33269]
 [184.8858 ]
 [184.79172]
 [184.76442]
 [185.03777]
 [185.444  ]] [[301.00001152 355.999993   348.00001085 355.00000681 421.9999924
  465.0000161  466.99998848 403.99999397 346.99999379 304.99998716
  335.99999132 339.99999783 317.99999288 362.00000277 348.00001085
  362.99998896 434.99999813 491.00002756 505.00001948 403.99999397
  359.00001333 310.00001073 337.00000838 359.99999952 342.00000109
  406.0000281  395.99998094 420.00002002 471.99998119 548.00004318


In [ ]:
# Calculate the RMSE

trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:, 0]))
print('Train : %.2f RMSE' % (trainScore))

Train : 76.40 RMSE


In [ ]:
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:, 0] ))
print('Test : %.2f RMSE' % (testScore))

Test : 242.08 RMSE
